<img src="https://learn.deeplearning.ai/assets/dlai-logo.png"></img>

# 🔬 Lab Introduction: Tool Use and Reflective Agents

In this lab, you will explore how AI agents can enhance research workflows by leveraging external tools and engaging in critical self-reflection. You'll learn how to build and integrate callable tools—such as web and academic search functions, and connect them to a language model using OpenAI's tool-calling API. Then, you’ll guide the agent to not only generate content but also **reflect** on its own output, improving the quality and depth of the final report. By the end of this lab, you will have implemented a mini agent capable of searching, reasoning, and publishing structured reports in HTML—laying the foundation for more advanced multi-step and autonomous AI systems.

### 🎯 Learning Objectives

By the end of this lab, you can:
- Chain steps into a research pipeline (**search → reflection → formatting**).
- Convert natural-language output into **styled HTML** suitable for sharing.

## ⚙️ Setup

This section:
- Loads environment variables
- Instantiates the OpenAI client

In [11]:
# ================================
# Standard library imports
# ================================
import json
import requests

# ================================
# Third-party imports
# ================================
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, HTML


# ================================
# Local / project imports
# ================================
import research_tools

# ================================
# Environment setup
# ================================
load_dotenv()  # Load environment variables from .env file
client = OpenAI()

## 🧰 Provided Tools

You’ll use two research helpers exposed in `research_tools`:
- **`arxiv_search_tool(query, max_results)`** – academic papers via arXiv API.
- **`tavily_search_tool(query, max_results, include_images)`** – general web search via Tavily.

## 🛠️ `arxiv_search_tool`

Searches arXiv and returns a list of papers with:
- `title`, `authors`, `published`, `summary`, `url`, and (if available) `link_pdf`.

Below, we run a quick test and print the results in a readable format.


In [12]:
# Test the arXiv search tool
results = research_tools.arxiv_search_tool("retrieval-augmented generation", max_results=3)

# Show formatted results
for i, paper in enumerate(results, 1):
    if "error" in paper:
        print(f"❌ Error: {paper['error']}")
    else:
        print(f"📄 Paper {i}")
        print(f"  Title     : {paper['title']}")
        print(f"  Authors   : {', '.join(paper['authors'])}")
        print(f"  Published : {paper['published']}")
        print(f"  URL       : {paper['url']}\n")


print("\n🧾 Raw Results:\n")
print(json.dumps(results, indent=2))

📄 Paper 1
  Title     : Generalized Baer and Generalized Quasi-Baer Rings of Skew Generalized
  Power Series
  Authors   : M. M. Hamam, R. E. Abdel-Khalek, R. M. Salem
  Published : 2024-05-06
  URL       : http://arxiv.org/abs/2405.03423v2

📄 Paper 2
  Title     : On generalized topological groups
  Authors   : Murad Hussain, Moiz Ud Din Khan, Cenap Özel
  Published : 2012-05-17
  URL       : http://arxiv.org/abs/1205.3915v1

📄 Paper 3
  Title     : Weighted spherical means generated by generalized translation and
  general Euler-Poisson-Darboux equation
  Authors   : Elina Shishkina
  Published : 2017-03-18
  URL       : http://arxiv.org/abs/1703.06340v1


🧾 Raw Results:

[
  {
    "title": "Generalized Baer and Generalized Quasi-Baer Rings of Skew Generalized\n  Power Series",
    "authors": [
      "M. M. Hamam",
      "R. E. Abdel-Khalek",
      "R. M. Salem"
    ],
    "published": "2024-05-06",
    "url": "http://arxiv.org/abs/2405.03423v2",
    "summary": "Let $R$ be a ring wi

## 🛠️ `tavily_search_tool`

Calls the Tavily API to fetch web results. Returns a list of dicts:
- `title`, `content`, `url` (and optional image URLs when `include_images=True`).

Run the cell to inspect sample output.

In [13]:
# Test the Tavily search tool
search_results = research_tools.tavily_search_tool("retrieval-augmented generation applications")
for item in search_results:
    print(item)

{'title': 'What is Retrieval Augmented Generation (RAG)? | Databricks', 'content': 'Retrieval augmented generation, or RAG, is an architectural approach that can improve the efficacy of large language model (LLM) applications by leveraging custom data. This is called retrieval augmented generation (RAG), as you would retrieve the relevant data and use it as augmented context for the LLM. With RAG architecture, organizations can deploy any LLM model and augment it to return relevant results for their organization by giving it a small amount of their data without the costs and time of fine-tuning or pretraining the model. * Using MLflow AI Gateway and Llama 2 to Build Generative AI Apps (Achieve greater accuracy using retrieval augmented generation (RAG) with your own data) Contact Databricks to schedule a demo and talk to someone about your LLM and retrieval augmented generation (RAG) projects', 'url': 'https://www.databricks.com/glossary/retrieval-augmented-generation-rag'}
{'title': '

## 🔗 Tool Mapping

We map tool names (strings) to the actual Python functions. This allows the model to call tools by name during tool-calling.

In [14]:
# Tool mapping
tool_mapping = {
    "tavily_search_tool": research_tools.tavily_search_tool,
    "arxiv_search_tool": research_tools.arxiv_search_tool,
}

---

### 🧠 Exercise 1: Tool-Calling Research Assistant

Implement `generate_research_report_with_tools(prompt_)` that:
1. Builds the conversation with a system message + user prompt.
2. Lets the model **call tools** (`arxiv_search_tool`, `tavily_search_tool`) as needed.
3. Executes tool calls, appends results, and continues the loop until a final answer.
4. Returns the **full message history** (including tool calls and responses).

> Aim for up to ~10 turns; stop when the assistant returns a final answer (no tool calls).


In [15]:
def generate_research_report_with_tools(prompt_: str, model: str = "gpt-4o") -> str:
    """
    Generates a research report using OpenAI's tool-calling with arXiv and Tavily tools.

    Args:
        prompt_ (str): The user prompt.
        model (str): OpenAI model name.

    Returns:
        str: Final assistant research report text.
    """
    messages = [
        {
            "role": "system",
            "content": (
                "You are a research assistant that can search the web and arXiv to write detailed, "
                "accurate, and properly sourced research reports.\n\n"
                "🔍 Use tools when appropriate (e.g., to find scientific papers or web content).\n"
                "📚 Cite sources whenever relevant. Do NOT omit citations for brevity.\n"
                "🌐 When possible, include full URLs (arXiv links, web sources, etc.).\n"
                "✍️ Use an academic tone, organize output into clearly labeled sections, and include "
                "inline citations or footnotes as needed.\n"
                "🚫 Do not include placeholder text such as '(citation needed)' or '(citations omitted)'."
            )
        },
        {"role": "user", "content": prompt_}
    ]

    functions = [research_tools.arxiv_tool_def, research_tools.tavily_tool_def]
    MAX_TURNS = 10
    final_text = None

    for _ in range(MAX_TURNS):
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=functions,
            tool_choice="auto",
            temperature=1,
        )

        msg = response.choices[0].message
        messages.append(msg)

        if not msg.tool_calls:
            final_text = msg.content
            print("✅ Final answer:")
            print(final_text)
            break

        for call in msg.tool_calls:
            tool_name = call.function.name
            args = json.loads(call.function.arguments)
            print(f"🛠️ {tool_name}({args})")

            try:
                tool_func = tool_mapping[tool_name]
                result = tool_func(**args)
            except Exception as e:
                result = {"error": str(e)}

            messages.append({
                "role": "tool",
                "tool_call_id": call.id,
                "name": tool_name,
                "content": json.dumps(result)
            })

    return final_text or ""



---

### 🧠 Exercise 2: Reflective Research Tool

Write `reflection(text_or_messages)` that produces a short, structured critique with:
- **Strengths**
- **Limitations**
- **Suggestions**
- **Opportunities**

Use an academic, concise tone. Accept either raw text or the `messages` list from the tool-calling step.


In [16]:
# %%
def reflection_and_rewrite(text_or_messages, model: str = "gpt-4o-mini", temperature: float = 0.3) -> dict:
    """
    Generates a structured reflection AND a revised research report.
    Accepts raw text OR the messages list returned by generate_research_report_with_tools.

    Returns:
        dict with keys:
          - "reflection": structured reflection text
          - "revised_report": improved version of the input report
    """
    # Extract assistant content if messages were passed
    if isinstance(text_or_messages, list):
        text = None
        for m in reversed(text_or_messages):
            role = m.get("role") if isinstance(m, dict) else getattr(m, "role", None)
            content = m.get("content") if isinstance(m, dict) else getattr(m, "content", None)
            if role == "assistant" and content:
                text = content
                break
        if not text:
            raise ValueError("No assistant text found in messages.")
    else:
        text = str(text_or_messages)

    # Ask for both reflection + rewritten report
    user_prompt = (
        "First, provide a structured reflection (Strengths, Limitations, Suggestions, Opportunities) "
        "on the following report.\n\n"
        "Then, write a revised version of the report that incorporates your suggestions, "
        "improves clarity, and strengthens academic tone.\n\n"
        f"Report:\n{text}"
    )

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are an academic reviewer and editor."},
            {"role": "user", "content": user_prompt},
        ],
        temperature=temperature,
    )

    # Expect the model to produce both sections in one response
    full_output = resp.choices[0].message.content.strip()

    return {
        "reflection": full_output,   # includes reflection
        "revised_report": full_output  # same output may include both parts
    }



### 🧠 Exercise 3: Publish as HTML

Use `generate_research_report_with_tools(prompt_, model="gpt-4o")` to let the model **call tools**, gather **evidence with citations**, and produce a **sourced answer**, then convert it to HTML.

**What it does (brief)**
1. Starts a chat with a **system policy** (use tools, cite URLs, be neutral) + your **prompt**.
2. Exposes two tools:
   - `arxiv_search_tool(query, max_results)`
   - `tavily_search_tool(query, max_results, include_images)`
3. Loops up to `MAX_TURNS`: executes tool calls, appends results; **stops** when the assistant replies with no more tool calls.
4. Normalizes messages to plain dicts (`role`, `content`, `tool_calls`) for easy downstream use.
5. Returns the **full message history** (dialogue, calls, results, final answer).

**Args**
- `prompt_` *(str)* – your research question.
- `model` *(str, default `gpt-4o`)*.

**Returns**
- `list` of message dicts including the final sourced answer.

In [17]:
# %%
def convert_report_to_html(text_or_messages, model: str = "gpt-4o") -> str:
    """
    Converts a plaintext research report into a styled HTML page using OpenAI.
    Accepts raw text OR the messages list from the tool-calling step.
    """
    # Inline extraction (sin helper)
    if isinstance(text_or_messages, list):
        text_report = None
        for m in reversed(text_or_messages):
            role = m.get("role") if isinstance(m, dict) else getattr(m, "role", None)
            content = m.get("content") if isinstance(m, dict) else getattr(m, "content", None)
            if role == "assistant" and content:
                text_report = content
                break
        if not text_report:
            raise ValueError("No assistant text found in messages.")
    else:
        text_report = str(text_or_messages)

    if not text_report:
        raise ValueError("Empty report text.")

    system_prompt = "You convert plaintext reports into full clean HTML documents."
    user_prompt = (
        "You are an expert technical writing assistant. "
        "Convert the following plaintext research report into a clean, structured HTML document. "
        "Include section headers, well-formatted paragraphs, inline links, and a clean readable layout. "
        "Ensure that all URLs are clickable and citation style is preserved.\n\n"
        "Respond ONLY with valid HTML (no explanation).\n\n"
        f"Report:\n{text_report}"
    )

    resp = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.5,
    )
    return resp.choices[0].message.content.strip()


### 🚀 End-to-End Pipeline

Run this cell to execute the full workflow:

1. Generate a research report (tools).
2. Reflect on the report.
3. Convert the report to HTML.

> You should see the rendered HTML below and two concise reflections in the console.

In [18]:
# 1) Research with tools
prompt_ = "Radio observations of recurrent novae"
preliminary_report = generate_research_report_with_tools(prompt_)
print("=== Research Report (preliminary) ===\n")
print(preliminary_report)

# 2) Reflection on the report (use the final TEXT to avoid ambiguity)
reflection_text = reflection_and_rewrite(preliminary_report)   # <-- pass text, not messages
print("=== Reflection on Report ===\n")
print(reflection_text['reflection'], "\n")
print("=== Revised Report ===\n")
print(reflection_text['revised_report'], "\n")


# 3) Convert the report to HTML (use the TEXT and correct function name)
html = convert_report_to_html(reflection_text['revised_report'])

print("=== Generated HTML (preview) ===\n")
print((html or "")[:600], "\n... [truncated]\n")

# 4) Display full HTML
display(HTML(html))


🛠️ arxiv_search_tool({'query': 'Radio observations of recurrent novae', 'max_results': 5})
✅ Final answer:
Radio observations of recurrent novae have provided valuable insights into the dynamics and composition of these intriguing astrophysical phenomena. Several studies highlight the various characteristics and behaviors of recurrent novae through different outbursts. Below is a summary of notable findings from recent research:

### Key Findings from Recent Research

1. **Review of Recurrent Novae**:
   - In a comprehensive review by Koji Mukai, it is noted that recurrent novae eruptions are often intensely observed across a broad spectrum, from radio to X-rays. A specific case, T Pyxidis (T Pyx), demonstrates a complex ejection of nova envelopes, suggesting that the envelope ejection in both recurrent and classical novae is a more intricate process than traditionally described [Mukai, 2014](http://arxiv.org/abs/1407.4526v1).

2. **2011 Outburst of T Pyx**:
   - Radio observations dur

---

## ✅ Wrap-Up

You built a mini research agent that can:
- 🔎 call tools (arXiv + Tavily),
- 🧠 reflect on its own output,
- 📰 publish a clean HTML report.

Great job!

### What to Submit
- Your notebook with Exercise 1–3 completed.

### Troubleshooting (quick)
- **Model/tool-call loop stalls?** Lower `MAX_TURNS` or print intermediate messages.
- **HTML looks odd?** Re-run conversion with a fresh assistant response.

**You’re done—nice work!** 🚀
